<a href="https://colab.research.google.com/github/ameyas1/CartPole_OpenAI/blob/master/cart_pole_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.


In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from collections import Counter

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# env = wrap_env(gym.make("CartPole-v0"))
env=gym.make('CartPole-v0')

In [7]:
print(env.action_space)

Discrete(2)


In [8]:
env.reset()

array([-0.02244801,  0.00608969,  0.01222607,  0.04048754])

In [0]:
def training():
  
  training_data=[]
  scores=[]
  accepted_scores=[]
  for _ in range(10000):
    
    score=0
    game_memory=[]
    prev_observation=[]
    for _ in range(500):
      action=env.action_space.sample()
      observation,reward,done,info=env.step(action)
      
      if(len(prev_observation))>0:
        game_memory.append([prev_observation,action])
        
      prev_observation=observation
      score+=reward
      if done:
        break
    env.reset()
    if(score>=50):
      accepted_scores.append(score)
      for data in game_memory:
        if data[1]==1:
          output=np.array([0,1])
        else:
          output=np.array([1,0])
        training_data.append([data[0],output])
    scores.append(score)
  
  training_data_save=np.array(training_data)
  np.save('saved',training_data_save)
  
  print('Average accepted score: ',np.mean(accepted_scores))
  print('Median accepted score: ',np.median(accepted_scores))
  print(Counter(accepted_scores))
  
  
  return training_data 

In [0]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.utils.data as data_utils

# define the NN architecture
class Net(nn.Module):
    def __init__(self,input):
        super(Net, self).__init__()
        self.input=input
        
        hidden_1 = 32
        hidden_2 = 64
        hidden_3=32
        hidden_4=16
        
        self.fc1 = nn.Linear(input, hidden_1)
        
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        
        self.fc4 = nn.Linear(hidden_3, hidden_4)
        
        self.fc5 = nn.Linear(hidden_4, 2)
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        
        
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        
        x = self.dropout(x)
        
        x =self.fc5(x)
        return x

In [11]:
train_data=training()

Average accepted score:  62.18051575931232
Median accepted score:  59.0
Counter({50.0: 32, 51.0: 24, 53.0: 22, 52.0: 18, 56.0: 17, 60.0: 16, 54.0: 15, 57.0: 15, 62.0: 14, 59.0: 14, 55.0: 13, 58.0: 13, 61.0: 11, 63.0: 9, 67.0: 9, 70.0: 8, 66.0: 7, 64.0: 7, 69.0: 7, 72.0: 6, 73.0: 6, 71.0: 6, 80.0: 5, 77.0: 5, 65.0: 5, 74.0: 4, 76.0: 4, 68.0: 4, 75.0: 3, 83.0: 3, 90.0: 3, 84.0: 2, 98.0: 2, 82.0: 2, 78.0: 2, 91.0: 2, 88.0: 2, 103.0: 1, 119.0: 1, 122.0: 1, 108.0: 1, 97.0: 1, 79.0: 1, 81.0: 1, 92.0: 1, 111.0: 1, 93.0: 1, 87.0: 1, 85.0: 1})


In [0]:
def train_model(train,model=False):
  X=[i[0] for i in train]
  X=np.array(X)
  y=[i[1] for i in train]
  y=np.array(y)
  
  X=torch.from_numpy(X)
  y=torch.from_numpy(y)
  
  train = data_utils.TensorDataset(X,y)
  train_loader = data_utils.DataLoader(train, batch_size=24, shuffle=True)
  
  n_epoch =10
  
  train_loss_min = np.Inf
  training_losses=[]

  
  if not model:
    model_1=Net(len(X[0]))
  else:
    model_1=Net(len(X[0]))
    model_1.load_state_dict(torch.load(model))
    
  if(torch.cuda.is_available()):
        model_1.cuda()
    
  print(model_1)
  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model_1.parameters(), lr=0.01)
  
  for epoch in range(n_epoch):
    train_loss = 0.0
    
    for x,y in train_loader:
      x,y = x.float(), y.long()
      
      model_1.train()
      if(torch.cuda.is_available()):
        x, y = x.cuda(), y.cuda()
      
      optimizer.zero_grad()
      output=model_1(x)
      loss=criterion(output,torch.max(y, 1)[1])
      loss.backward()
      optimizer.step()
      
      train_loss += loss.item()*x.size(0)
      
      
    model_1.eval()
      
    train_loss = train_loss/len(train_loader.dataset)
    
    training_losses.append(train_loss)
   
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
          epoch+1, 
          train_loss
          ))
  
  
    if train_loss <= train_loss_min:
      print('training loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(train_loss_min,train_loss))
      torch.save(model_1.state_dict(), 'model.pt')
      train_loss_min = train_loss
  
  
  return model_1
      
      
    
  
  
  
    
#   model_1.fit(X,y,epochs=3)


In [0]:
def game(model_cp):
  
  
  scores=[]
  choices=[]
  for each_game in range(10):
    env=wrap_env(gym.make('CartPole-v0'))
    env.render()
    score=0
    game_memory=[]
    prev_obs=[]
    env.reset()
    
  
    while True:
      if(len(prev_obs)==0):
        action=env.action_space.sample()
      elif(len(prev_obs)>0):
        
        prev_obs_n=np.array([prev_obs])
        prev_obs_n=torch.from_numpy(prev_obs_n)
        prev_obs_n=prev_obs_n.float()
        
        if(torch.cuda.is_available()):
          prev_obs_n = prev_obs_n.cuda()
      
      
        pred=model_cp(prev_obs_n)
        _, action = torch.max(pred, 1)
        action=action.cpu().squeeze().numpy()
      
      choices.append(action) 
      observation,reward,done,info=env.step(action)
      prev_obs=observation
      if(action==1):
        game_memory.append([prev_obs,[0,1]])
      else:
        game_memory.append([prev_obs,[1,0]])
      score+=reward
      if done:
          env.close()
          show_video()

          scores.append(score)
          break
    
    
  print('average score',sum(scores)/len(scores))
  print('Choice 1:{}, choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
  print(scores)

  
  return game_memory

In [14]:
train_data=training()
model_cp=train_model(train_data)

Average accepted score:  60.687150837988824
Median accepted score:  57.0
Counter({50.0: 39, 55.0: 29, 52.0: 25, 53.0: 22, 54.0: 21, 51.0: 20, 60.0: 17, 57.0: 16, 56.0: 15, 63.0: 12, 59.0: 12, 64.0: 12, 58.0: 12, 62.0: 9, 61.0: 8, 73.0: 7, 67.0: 7, 70.0: 6, 66.0: 6, 68.0: 6, 65.0: 6, 72.0: 5, 69.0: 5, 76.0: 4, 83.0: 4, 74.0: 3, 88.0: 3, 71.0: 3, 78.0: 2, 102.0: 2, 85.0: 2, 79.0: 2, 77.0: 2, 106.0: 2, 81.0: 2, 96.0: 1, 108.0: 1, 84.0: 1, 75.0: 1, 97.0: 1, 87.0: 1, 103.0: 1, 89.0: 1, 146.0: 1, 80.0: 1})
Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=2, bias=True)
  (dropout): Dropout(p=0.1)
)
Epoch: 1 	Training Loss: 0.679609
training loss decreased (inf --> 0.679609).  Saving model ...
Epoch: 2 	Training Loss: 0.674805
training loss decreased (0.6796

In [15]:
model_cp.load_state_dict(torch.load('model.pt'))

game_memory=game(model_cp)

average score 200.0
Choice 1:0.5, choice 0:0.5
[200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0]
